In [ ]:
# This notebook consists of actions for preparation for Horizon tests
# It creates a fake cube with horizon and runs tests
import os
import sys
from datetime import date
import shutil
import warnings
import nbformat
import numpy as np
from py.path import local

warnings.filterwarnings("ignore")

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

from seismiqb.src.synthetic import generate_synthetic
from seismiqb.src.geometry.export import make_segy_from_array
from seismiqb import Horizon, Field
from seismiqb.batchflow.utils_notebook import run_notebook
from seismiqb.tests.utils import extract_traceback

In [ ]:
# # Parameters:

# # Workspace
# DATESTAMP = date.today().strftime("%Y-%m-%d")
# NOTEBOOKS_DIR = './'
# SAVING_DIR = './horizon_test_files'

# # Synthetic creation
# SYNTHETIC_MODE = True
# CUBE_PATH = None
# HORIZON_PATH = None
# CUBE_SHAPE = (500, 500, 200)
# GRID_SHAPE = (10, 10)
# SEED = 42

# # Visualization
# FIGSIZE = (12, 7)
# SHOW_FIGURES = False # Whether to show additional figures

# # Execution
# DROP_EXTRA_FILES = True
# SHOW_TEST_ERROR_INFO = True
# GITHUB_MODE = False

# Preparation

Create directories for files and create a fake cube with horizons and save cube and one horizon.

**Storage structure:**
___



**geometry_test_files** (tests root directory)

&emsp;├── **tmp** (directory with temporary files)

&emsp;│&emsp;&emsp;├── **test_cube.sgy**
    
&emsp;│&emsp;&emsp;└── **test_horizon**

&emsp;├── **horizon_attributes_test_out.ipynb** (tests notebooks outputs)

&emsp;├── **horizon_base_test_out.ipynb**

&emsp;├── **horizon_manipulations_test_out.ipynb**

&emsp;├── **horizon_merge_test_out.ipynb**

&emsp;└── **message.txt** (file with output message)

In [ ]:
# Storage preparation:

if GITHUB_MODE:
    _ = SAVING_DIR.mkdir('tmp')

else:
    # Clear and recreate workspace
    try:
        shutil.rmtree(SAVING_DIR)
    except OSError as e:
        print(f"Can't delete the directory {SAVING_DIR} : {e.strerror}")

    if not os.path.exists(SAVING_DIR):
        os.makedirs(SAVING_DIR)
        os.makedirs(os.path.join(SAVING_DIR, 'tmp'))


msg = DATESTAMP + '\n\n'
is_all_OK = True

In [ ]:
%%time
# Synthetic data creation

if SYNTHETIC_MODE:
    CUBE_PATH = os.path.join(SAVING_DIR, 'tmp/test_cube.sgy')
    HORIZON_PATH = os.path.join(SAVING_DIR, 'tmp/test_horizon')

    # Create a cube and save it
    synt3d, hors, _ = generate_synthetic(shape=CUBE_SHAPE, grid_shape=GRID_SHAPE,
                                         num_reflections=60,
                                         geobodies_format=('heights', 'mask'), seed=SEED)

    # Create zero traces in the cube (for testing filetring)
    FILL_VALUE = -999999
    points = (
        slice(0, CUBE_SHAPE[0]//10, None),
        slice(CUBE_SHAPE[0]//10, CUBE_SHAPE[0]//5, None),
        slice(None)
    )
    synt3d[points] = FILL_VALUE

    make_segy_from_array(synt3d, CUBE_PATH, zip_segy=False)

    # Check cube
    field = Field(CUBE_PATH)

    assert np.allclose(field.geometry[:, :, :], synt3d)

    # Choose one horizon and save it
    horizons = [Horizon(hor, field, 'synthetic_' + str(i)) for i, hor in enumerate(hors)]
    horizon = horizons[0]

    horizon.dump(HORIZON_PATH)

    # Check horizon
    opened_horizon = Horizon(HORIZON_PATH, field=field)

    assert np.array_equal(horizon.matrix, opened_horizon.matrix)
    assert np.array_equal(horizon.points, opened_horizon.points)

    opened_horizon.filter()
    opened_horizon.show(show=SHOW_FIGURES, figsize=FIGSIZE)

    msg += 'Synthetic data was successfully created.\n'

print(msg)

# Run tests

In [ ]:
def exit_message(test_name, exec_info, out_path_ipynb, is_all_OK):
    " Construct exit message for test and drop extra file."
    if exec_info is True:
        msg = f'Horizon {test_name} tests were executed successfully.\n'

        if DROP_EXTRA_FILES:
            print(out_path_ipynb)
            try:
                os.remove(out_path_ipynb)
            except OSError as e:
                print(f"Can't delete the file: {out_path_ipynb} : {e.strerror}")

    else:
        msg = f'Horizon {test_name} tests execution failed.\n'
        is_all_OK = False

        if SHOW_TEST_ERROR_INFO:
            # Add error traceback into the message
            msg += extract_traceback(path_ipynb=out_path_ipynb)

    return msg, is_all_OK

In [ ]:
%%time
# Base test
out_path_ipynb = os.path.join(SAVING_DIR, f'horizon_base_test_out_{DATESTAMP}.ipynb')

exec_info = run_notebook(
    path=os.path.join(NOTEBOOKS_DIR, 'horizon_base_test.ipynb'),
    nb_kwargs={
        'CUBE_PATH': CUBE_PATH,
        'HORIZON_PATH': HORIZON_PATH,
        'TMP_DIR': os.path.join(SAVING_DIR, 'tmp'),
        'FIGSIZE': FIGSIZE,
        'SHOW_FIGURES': SHOW_FIGURES
    },
    insert_pos=1,
    out_path_ipynb=out_path_ipynb,
    display_links=False
)

current_msg, is_all_OK = exit_message(test_name='base', exec_info=exec_info,
                                      out_path_ipynb=out_path_ipynb, is_all_OK=is_all_OK)

print(current_msg)

msg += current_msg

In [ ]:
# Will be added later in separate PR
# %%time
# # Merge test
# out_path_ipynb = os.path.join(SAVING_DIR, f'horizon_merge_test_out_{DATESTAMP}.ipynb')

# exec_info = run_notebook(
#     path=os.path.join(NOTEBOOKS_DIR, 'horizon_merge_test.ipynb'),
#     nb_kwargs={
#         'CUBE_PATH': CUBE_PATH,
#         'HORIZON_PATH': HORIZON_PATH,
#         'FIGSIZE': FIGSIZE,
#         'SHOW_FIGURES': SHOW_FIGURES
#     },
#     insert_pos=1,
#     out_path_ipynb=out_path_ipynb,
#     display_links=False
# )

# current_msg, is_all_OK = exit_message(test_name='merge', exec_info=exec_info,
#                                       out_path_ipynb=out_path_ipynb, is_all_OK=is_all_OK)

# print(current_msg)

# msg += current_msg

In [ ]:
%%time
# Manipulations test
out_path_ipynb = os.path.join(SAVING_DIR, f'horizon_manipulations_test_out_{DATESTAMP}.ipynb')

exec_info = run_notebook(
    path=os.path.join(NOTEBOOKS_DIR, 'horizon_manipulations_test.ipynb'),
    nb_kwargs={
        'CUBE_PATH': CUBE_PATH,
        'HORIZON_PATH': HORIZON_PATH,
        'FIGSIZE': FIGSIZE,
        'SHOW_FIGURES': SHOW_FIGURES
    },
    insert_pos=1,
    out_path_ipynb=out_path_ipynb,
    display_links=False
)

current_msg, is_all_OK = exit_message(test_name='manipulations', exec_info=exec_info,
                                      out_path_ipynb=out_path_ipynb, is_all_OK=is_all_OK)

print(current_msg)

msg += current_msg

In [ ]:
%%time
# Attributes test
out_path_ipynb = os.path.join(SAVING_DIR, f'horizon_attributes_test_out_{DATESTAMP}.ipynb')

exec_info = run_notebook(
    path=os.path.join(NOTEBOOKS_DIR, 'horizon_attributes_test.ipynb'),
    nb_kwargs={
        'CUBE_PATH': CUBE_PATH,
        'HORIZON_PATH': HORIZON_PATH,
        'FIGSIZE': FIGSIZE
    },
    insert_pos=1,
    out_path_ipynb=out_path_ipynb,
    display_links=False
)

current_msg, is_all_OK = exit_message(test_name='attributes', exec_info=exec_info,
                                      out_path_ipynb=out_path_ipynb, is_all_OK=is_all_OK)

print(current_msg)

msg += current_msg

# Exit

In [ ]:
# Drop synthetic data if needed
if is_all_OK and DROP_EXTRA_FILES and SYNTHETIC_MODE:

    for file_path in [CUBE_PATH, HORIZON_PATH]:
        try:
            os.remove(file_path)
        except OSError as e:
            print(f"Can't delete the file: {file_path} : {e.strerror}")

if is_all_OK:
    msg += 'Horizon tests were executed successfully.'

# Save message
with open(os.path.join(SAVING_DIR, f'message_{DATESTAMP}.txt'), "w") as outfile:
    outfile.write(msg)

print(msg)